In [0]:
%sql
SELECT current_catalog(), current_schema();


current_catalog(),current_schema()
workspace,default


In [0]:
%sql
SELECT * FROM ecommerce.silver.events;


event_id,user_id,event_time


In [0]:
%sql
DESCRIBE TABLE ecommerce.silver.events;


col_name,data_type,comment
event_id,string,null
user_id,string,null
event_time,timestamp,null


In [0]:
%sql
USE CATALOG ecommerce;
USE SCHEMA silver;


In [0]:
%sql
SELECT * FROM events;


event_id,user_id,event_time


In [0]:
%sql
SELECT current_catalog(), current_schema();


current_catalog(),current_schema()
ecommerce,silver


In [0]:
%sql
EXPLAIN FORMATTED
SELECT *
FROM events
WHERE event_type = 'purchase';


plan
Error occurred during query planning:
"[UNRESOLVED_COLUMN.WITH_SUGGESTION] A column, variable, or function parameter with name `event_type` cannot be resolved. Did you mean one of the following? [`event_time`, `event_id`, `user_id`]. SQLSTATE: 42703; line 4 pos 6;"
'Project [*]
+- 'Filter ('event_type = purchase)
+- SubqueryAlias ecommerce.silver.events
"+- Relation ecommerce.silver.events[event_id#13359,user_id#13360,event_time#13361] parquet"


In [0]:
%sql
SHOW TABLES LIKE 'events';


database,tableName,isTemporary
silver,events,false


In [0]:
%sql
DESCRIBE EXTENDED events;


col_name,data_type,comment
event_id,string,null
user_id,string,null
event_time,timestamp,null
,,
# Detailed Table Information,,
Catalog,ecommerce,
Database,silver,
Table,events,
Created Time,Fri Jan 16 17:35:03 UTC 2026,
Last Access,UNKNOWN,


In [0]:
%sql
EXPLAIN FORMATTED
SELECT *
FROM events
WHERE event_type = 'purchase'
  AND event_date = '2026-01-15';


plan
Error occurred during query planning:
"[UNRESOLVED_COLUMN.WITH_SUGGESTION] A column, variable, or function parameter with name `event_type` cannot be resolved. Did you mean one of the following? [`event_time`, `event_id`, `user_id`]. SQLSTATE: 42703; line 4 pos 6;"
'Project [*]
+- 'Filter (('event_type = purchase) AND ('event_date = 2026-01-15))
+- SubqueryAlias ecommerce.silver.events
"+- Relation ecommerce.silver.events[event_id#13446,user_id#13447,event_time#13448] parquet"


In [0]:
%sql
OPTIMIZE event
ZORDER BY (user_id, product_id);


---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-4966268229005466>, line 1
----> 1 get_ipython().run_cell_magic('sql', '', 'OPTIMIZE event\nZORDER BY (user_id, product_id);\n')

File /databricks/python/lib/python3.12/site-packages/IPython/core/interactiveshell.py:2541, in InteractiveShell.run_cell_magic(self, magic_name, line, cell)
   2539 with self.builtin_trap:
   2540     args = (magic_arg_s, cell)
-> 2541     result = fn(*args, **kwargs)
   2543 # The code below prevents the output from being displayed
   2544 # when using magics with decorator @output_can_be_silenced
   2545 # when the last Python token in the expression is a ';'.
   2546 if getattr(fn, magic.MAGIC_OUTPUT_CAN_BE_SILENCED, False):

File /databricks/python_shell/lib/dbruntime/sql_magic/sql_magic.py:194, in SqlMagic.sql(self, line, cell)
    188 except BaseException as e:
    189     self.driver_activ

In [0]:
%sql
EXPLAIN
SELECT *
FROM events
WHERE user_id = 12345;


plan
"== Physical Plan == LocalTableScan , [event_id#13578, user_id#13579, event_time#13580] == Photon Explanation == Photon does not fully support the query because: Unsupported node: LocalTableScan , [event_id#13578, user_id#13579, event_time#13580]. Reference node: LocalTableScan , [event_id#13578, user_id#13579, event_time#13580]"


In [0]:
%sql
EXPLAIN
SELECT *
FROM events_part
WHERE user_id = 12345;


plan
Error occurred during query planning:
[TABLE_OR_VIEW_NOT_FOUND] The table or view `events_part` cannot be found. Verify the spelling and correctness of the schema and catalog.
"If you did not qualify the name with a schema, verify the current_schema() output, or qualify the name with the correct schema and catalog."
To tolerate the error on drop use DROP VIEW IF EXISTS or DROP TABLE IF EXISTS. SQLSTATE: 42P01; line 3 pos 5;
'Project [*]
+- 'Filter ('user_id = 12345)
"+- 'UnresolvedRelation [events_part], [], false"


In [0]:
import time

start = time.time()
spark.sql("""
SELECT *
FROM events_part
WHERE user_id = 12345
""").count()

print(f"Time: {time.time() - start:.2f}s")


---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-4966268229005469>, line 4
      1 import time
      3 start = time.time()
----> 4 spark.sql("""
      5 SELECT *
      6 FROM events_part
      7 WHERE user_id = 12345
      8 """).count()
     10 print(f"Time: {time.time() - start:.2f}s")

File /databricks/python/lib/python3.12/site-packages/pyspark/sql/connect/session.py:875, in SparkSession.sql(self, sqlQuery, args, **kwargs)
    872         _views.append(SubqueryAlias(df._plan, name))
    874 cmd = SQL(sqlQuery, _args, _named_args, _views)
--> 875 data, properties, ei = self.client.execute_command(cmd.command(self._client))
    876 if "sql_command_result" in properties:
    877     df = DataFrame(CachedRelation(properties["sql_command_result"]), self)

File /databricks/python/lib/python3.12/site-packages/pyspark/sql/connect/client/core.py:1556, in SparkConnectClient.ex

In [0]:
df = spark.table("events_part")
df.cache()
df.count()  # materialize


---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-4966268229005470>, line 2
      1 df = spark.table("events_part")
----> 2 df.cache()
      3 df.count()

File /databricks/python/lib/python3.12/site-packages/pyspark/sql/connect/dataframe.py:2093, in DataFrame.cache(self)
   2092 def cache(self) -> ParentDataFrame:
-> 2093     return self.persist()

File /databricks/python/lib/python3.12/site-packages/pyspark/sql/connect/dataframe.py:2100, in DataFrame.persist(self, storageLevel)
   2095 def persist(
   2096     self,
   2097     storageLevel: StorageLevel = (StorageLevel.MEMORY_AND_DISK_DESER),
   2098 ) -> ParentDataFrame:
   2099     relation = self._plan.plan(self._session.client)
-> 2100     self._session.client._analyze(
   2101         method="persist", relation=relation, storage_level=storageLevel
   2102     )
   2103     return self

File /databricks/python/lib/p